# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [1]:
#ваш код
import pandas as pd
import numpy as np

df = pd.read_csv('Advertising.csv')
df

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [2]:
#ваш код

df.info()
# пропусков нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [3]:
#ваш код
# Столбец 'Unnamed' не несёт никакой смысловой нагрузки, можем его удалить
# df.drop(columns='Unnamed: 0', inplace=True)

# Зададим предикторы
X = np.array(df[['TV', 'radio', 'newspaper']])

# Зададим таргет
y = np.array(df['sales'])

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [4]:
import numpy as np
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [5]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм:

<a href="https://ibb.co/Th3BQFn"><img src="https://i.ibb.co/DK2DBS6/zascas.jpg" alt="zascas" border="0"></a>

Примечание: 1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [6]:
#ваш код
def coord_desc(X, y, iter):
    w = np.zeros(X.shape[1])

    for i in range(iter):
        for j in range(len(w)):
            r = y - X @ w   # внести этот шаг во внутренний цикл будет более верно, т.к. иначе при определении w[j] с 2 по len(w) не будут учитываться ранее рассчитанные веса в пределах одной итерации
            r_no_j = r + X[:, j] * w[j]
            w[j] = X[:, j] @ r_no_j
            r = r - X[:, j] * w[j]

    return w

print(f'Веса модели: {coord_desc(X=X, y=y, iter=1000)}')

Веса модели: [ 41.56217205 110.13144155  73.52860638  -0.55006384]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [7]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[ 41.56217205 110.13144155  73.52860638  -0.55006384]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [8]:
#ваш код
# Заново зададим датафрейм, предикторы (X)
df = pd.read_csv('Advertising.csv')
X = np.array(df[['TV', 'radio', 'newspaper']])

# Отмасштабируем полученные данные
std = np.std(X, axis=0)
std[std == 0] = 1
X = (X - np.mean(X, axis=0)) / std

**Добавим единичный столбец**

In [9]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [10]:
#ваш код
def mse_error(real, pred):
    mse = np.mean(np.square(real - pred))
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [11]:
#ваш код
# Зададим предсказание средним значением
fake_pred = np.mean(y)

# Считаем mse
print(f'MSE on naive prediction: {mse_error(y, fake_pred)}')

MSE on naive prediction: 27.085743750000002


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [12]:
#ваш код
def lin_pred(X, w):
    y_pred = X @ w
    return y_pred

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [13]:
#ваш код
def stoch_grad_step(X, y, w, train_ind, eta):
    diff_dir = np.array(X[train_ind] * 2 * (X[train_ind] @ w - y[train_ind]) / len(y))
    step = w - np.array(diff_dir) * eta
    return step

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [14]:
# ваш код
def stochastic_gradient_descent(X, y, w, eta, max_iter, min_dist):
    # Расстояние между векторами весов на соседних итерациях
    curr_dist = np. inf
    # Список для фиксации ошибок
    error = []
    # Счётчик итераций
    curr_iter = 0
    # Для удобства проверки результатов зададим seed
    np.random.seed(42)

    while curr_dist > min_dist and curr_iter < max_iter:
        # Задаём случайную строку для итерации в диапазоне от 0 до количества строк предикторов
        random_ind = np.random.randint(X.shape[0])
        # Считаем текущий шаг стохастического спуска
        new_w = stoch_grad_step(X=X, y=y, w=w, train_ind=random_ind, eta=eta)
        # Считаем предсказанное значение с новыми весами
        y_pred = lin_pred(X=X, w=new_w)
        # Записываем текующую среднеквадратичную ошибку
        error.append(mse_error(y, y_pred))
        # Увеличиваем показатель счётчика
        curr_iter += 1
        #Расстояние между весами
        curr_dist = np.linalg.norm(new_w - w)
        # Перезаписываем веса
        w = new_w
        
    return w, error

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [15]:
# ваш код
# Зададим количество итераций, вектро начальных весов, минимальное евклидово расстояние и eta
iter = 10 ** 5
w = np.zeros(X.shape[1])
min_dist = 1e-8
eta = 0.01

# Запустим функцию
vect, mse = stochastic_gradient_descent(X=X, y=y, w=w, eta=eta, max_iter=iter, min_dist=min_dist)

**Выведите вектор весов, к которому сошелся метод.**

In [16]:
# ваш код
print(f'Вектор весов: {vect}')

Вектор весов: [ 1.40190566e+01  3.91069256e+00  2.78209808e+00 -8.10462217e-03]


**Выведите среднеквадратичную ошибку на последней итерации.**

In [17]:
# ваш код
print(f'Среднеквадратичная ошибка на последней итерации: {mse[-1]}')

Среднеквадратичная ошибка на последней итерации: 2.7844125884067035
